In [0]:
import h5py
import numpy as np
import tensorflow as tf
tf.keras.backend.set_floatx('float32')

class Encoder(tf.keras.Model):
  def __init__(self, encoder_units, conv1d_units, batch_size):
    super(Encoder, self).__init__()
    self.batch_size = batch_size
    self.encoder_units = encoder_units
    self.weights_dir = "/content/drive/My Drive/Colab/Research - EE5003/Phase 1 - Comparison of forecasting technique/Model Weights/encoder.h5py"
    self.lstm = tf.keras.layers.LSTM(encoder_units,
                                     return_sequences=True,
                                     return_state=True)
    
  def call(self, x, state_h, state_c):
    # output - (batch_size, timesteps_in, encoder_units)
    # state_h, state_c - (batch_size, encoder_units)
    output, state_h, state_c = self.lstm(x, initial_state=[state_h, state_c])
    return output, state_h, state_c
  
  def initialize_state(self):
    state_h = tf.zeros((self.batch_size, self.encoder_units))
    state_c = tf.zeros((self.batch_size, self.encoder_units))
    return state_h, state_c

class Decoder(tf.keras.Model):
  def __init__(self, decoder_units, conv1d_units):
    super(Decoder, self).__init__()
    self.weights_dir = "/content/drive/My Drive/Colab/Research - EE5003/Phase 1 - Comparison of forecasting technique/Model Weights/decoder.h5py"
    self.conv1d_s = tf.keras.layers.Conv1D(conv1d_units, kernel_size=1, 
                                           strides=1, padding="same", 
                                           activation="relu")
    self.conv1d_h = tf.keras.layers.Conv1D(conv1d_units, kernel_size=1, 
                                           strides=1, padding="same", 
                                           activation="relu")
    self.conv1d_v = tf.keras.layers.Conv1D(1, kernel_size=1, 
                                           strides=1, padding="same", 
                                           activation="sigmoid")     
    self.conv1d_output = tf.keras.layers.Conv1D(1, kernel_size=1, 
                                                strides=1, padding="same", 
                                                activation="linear")    
    self.lstm = tf.keras.layers.LSTM(decoder_units,
                                     return_sequences=True,
                                     return_state=True)
    self.tanh = tf.keras.layers.Activation("tanh")
    self.softmax = tf.keras.layers.Softmax(axis=1)

  def call(self, dec_input, enc_output, state_h, state_c):
    """
    1. Use the hidden and cell state from encoder's last state as the initial 
    state of decoder lstm
    2. score(t) = w_v * (tanh(w_s * enc_output + w_h * state_h(t)))
    3. attention(t_i) = exp(score(t_i)) / sum(exp(score(t_i)))
    4. content_vector = sum(attention * enc_output) - sum across all timesteps
    """

    # dec_hidden - (batch_size, encoder_units)
    dec_hidden = [state_h, state_c]

    # expand state_h to match enc_output dimension
    # state_h_time, state_c_time - (batch_size, 1, encoder_units)
    # state_hc_time - (batch_size, 1, 2 * encoder_units)
    state_h_time = tf.expand_dims(state_h, axis=1)
    state_c_time = tf.expand_dims(state_c, axis=1)
    state_hc_time = tf.concat([state_h_time, state_c_time], axis=-1)

    # additive score
    # score_s - (batch_size, timesteps_in, conv1d_units)
    # score_hc - (batch_size, 1, conv1d_units)
    # score - (batch_size, timesteps_in, 1)
    score_s = self.conv1d_s(enc_output)
    score_h = self.conv1d_h(state_hc_time)
    score = self.tanh(score_s + score_h)
    score = self.conv1d_v(score)

    # attention weights - (batch_size, timesteps_in, 1)
    attention_weights = self.softmax(score)
  
    # content vector - before sum (batch_size, timesteps_in, encoder_units)
    # content vector - after sum (batch_size, encoder_units)
    # content vector - after expansion (batch_size, 1, encoder_units)
    # content vector - after concatenation (batch_size, 1, encoder_units + 1)
    content_vector = attention_weights * enc_output
    content_vector = tf.reduce_sum(content_vector, axis=1)
    content_vector = tf.expand_dims(content_vector, axis=1)
    content_vector = tf.concat([content_vector, dec_input], axis=-1) 
    
    # compute the output and hidden and cell state
    # dec_output - (batch_size, 1, decoder_units)
    # state_h, state_c - (batch_size, encoder_units)
    dec_output, state_h, state_c = self.lstm(content_vector, initial_state=dec_hidden)

    # output - (batch_size, 1, 1) 
    output = self.conv1d_output(dec_output)

    return output, attention_weights, state_h, state_c